<h1>Building random forest models for Co-occurrence between smells and bugs</h1>

In [1]:
import pandas as pd

In [2]:
#read dataset ready for processing
df=pd.read_csv("RQ3_data.csv")

In [3]:
df.shape

(29373, 24)

In [4]:
df.columns

Index(['FEAR_OF_THE_UNKNOWN', 'IMPLICIT_COLUMNS', 'AMBIGUOUS_GROUPS',
       'RANDOM_SELECTION', 'AntiSingleton', 'BaseClassKnowsDerivedClass',
       'BaseClassShouldBeAbstract', 'Blobs', 'ClassDataShouldBePrivate',
       'ComplexClass', 'FunctionalDecomposition', 'LargeClass', 'LazyClass',
       'LongMethod', 'LongParameterList', 'ManyFieldAttributesButNotComplex',
       'MessageChains', 'RefusedParentBequest', 'SpaghettiCode',
       'SpeculativeGenerality', 'SwissArmyKnife', 'TraditionBreaker',
       'in_bug_fix', 'in_bug_induce'],
      dtype='object')

In [5]:
df['in_bug_induce'].value_counts()

0    26158
1     3215
Name: in_bug_induce, dtype: int64

<h2>Buiding model and evaluating feature contribution on in_bug_induce</h2>

In [6]:
x_train= df[['FEAR_OF_THE_UNKNOWN', 'IMPLICIT_COLUMNS', 'AMBIGUOUS_GROUPS',
       'RANDOM_SELECTION', 'AntiSingleton', 'BaseClassKnowsDerivedClass',
       'BaseClassShouldBeAbstract', 'Blobs', 'ClassDataShouldBePrivate',
       'ComplexClass', 'FunctionalDecomposition', 'LargeClass', 'LazyClass',
       'LongMethod', 'LongParameterList', 'ManyFieldAttributesButNotComplex',
       'MessageChains', 'RefusedParentBequest', 'SpaghettiCode',
       'SpeculativeGenerality', 'SwissArmyKnife', 'TraditionBreaker']]
x_train.columns

Index(['FEAR_OF_THE_UNKNOWN', 'IMPLICIT_COLUMNS', 'AMBIGUOUS_GROUPS',
       'RANDOM_SELECTION', 'AntiSingleton', 'BaseClassKnowsDerivedClass',
       'BaseClassShouldBeAbstract', 'Blobs', 'ClassDataShouldBePrivate',
       'ComplexClass', 'FunctionalDecomposition', 'LargeClass', 'LazyClass',
       'LongMethod', 'LongParameterList', 'ManyFieldAttributesButNotComplex',
       'MessageChains', 'RefusedParentBequest', 'SpaghettiCode',
       'SpeculativeGenerality', 'SwissArmyKnife', 'TraditionBreaker'],
      dtype='object')

In [7]:
y_train=df['in_bug_induce']
y_train.value_counts()

0    26158
1     3215
Name: in_bug_induce, dtype: int64

In [8]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(1.0)
x_rus, y_rus= sm.fit_sample(x_train, y_train)
y_rus.value_counts()

1    26158
0    26158
Name: in_bug_induce, dtype: int64

In [9]:
#Import classifier, random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold

In [10]:
#run classifier 10 times using cross validation to try different models and take the median feature importance 
#on all models
kf = KFold(n_splits=10)
clf= RandomForestClassifier(n_estimators=100)
res_df=[]
res_df.append(x_rus.columns)
for train, _ in kf.split(x_rus, y_rus):
    clf.fit(x_rus.iloc[train, :],y_rus.iloc[train])
    res=[]
    
    for ind, val in enumerate(clf.feature_importances_):
        res.append(val*100)
    res_df.append(res)
res_frame=pd.DataFrame(res_df)
res_frame.head(100)

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,FEAR_OF_THE_UNKNOWN,IMPLICIT_COLUMNS,AMBIGUOUS_GROUPS,RANDOM_SELECTION,AntiSingleton,BaseClassKnowsDerivedClass,BaseClassShouldBeAbstract,Blobs,ClassDataShouldBePrivate,ComplexClass,...,LazyClass,LongMethod,LongParameterList,ManyFieldAttributesButNotComplex,MessageChains,RefusedParentBequest,SpaghettiCode,SpeculativeGenerality,SwissArmyKnife,TraditionBreaker
1,5.99857,11.7493,0.184562,0,8.33421,0,1.05381,0,0,37.8157,...,3.14295,9.82719,9.4759,0.434465,0,1.69824,9.34461,0,0.909914,0
2,9.02996,5.08655,0.902527,0,7.33828,0,1.05329,0,0,38.8841,...,6.48416,10.3832,10.3515,0.401824,0,1.37289,7.75285,0,0.932487,0
3,8.82064,5.58052,0.984156,0,8.48529,0,0.967892,0,0,38.958,...,4.90258,11.3816,9.34196,0.42492,0,1.50219,7.73495,0,0.882692,0
4,7.76198,5.19849,0.7128,0,8.20029,0,0.619293,0,0,40.6983,...,7.6333,9.4887,11.8019,0.168386,0,0.781335,6.60549,0,0.329651,0
5,7.18377,5.33345,0.813256,0,3.66786,0,0.333534,0,0,45.5512,...,7.24467,11.131,12.7383,0.218718,0,1.47061,3.93387,0,0.358554,0
6,7.15987,5.79609,0.775126,0,5.24431,0,0.517858,0,0,44.608,...,7.59561,9.64784,11.8242,0.354288,0,1.34935,4.40597,0,0.687933,0
7,7.18851,4.78377,0.777935,0,6.02186,0,0.583516,0,0,44.3827,...,6.20612,11.1321,10.2778,0.318337,0,1.26717,6.40893,0,0.607456,0
8,6.5951,5.20306,0.897677,0,5.88245,0,0.702236,0,0,46.1359,...,6.00296,11.2883,9.12352,0.301402,0,1.40495,5.82814,0,0.593643,0
9,6.48207,5.29509,0.995626,0,6.41155,0,0.702659,0,0,45.4045,...,5.81934,10.8519,9.60574,0.321477,0,1.41529,6.06937,0,0.578553,0


In [11]:
res_frame.head()
pd.options.display.float_format = '{:,.2f}'.format
res_frame.median().sort_values(ascending=False) #get the median of percentage feature contribution
res_frame.describe()
res_frame_t = res_frame.T
res_frame_t.head(25)

,0,1,2,3,4,5,6,7,8,9,10
0,FEAR_OF_THE_UNKNOWN,6.00,9.03,8.82,7.76,7.18,7.16,7.19,6.60,6.48,7.47
1,IMPLICIT_COLUMNS,11.75,5.09,5.58,5.20,5.33,5.80,4.78,5.20,5.30,5.30
2,AMBIGUOUS_GROUPS,0.18,0.90,0.98,0.71,0.81,0.78,0.78,0.90,1.00,0.88
3,RANDOM_SELECTION,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,AntiSingleton,8.33,7.34,8.49,8.20,3.67,5.24,6.02,5.88,6.41,6.96
5,BaseClassKnowsDerivedClass,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,BaseClassShouldBeAbstract,1.05,1.05,0.97,0.62,0.33,0.52,0.58,0.70,0.70,0.66
7,Blobs,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,ClassDataShouldBePrivate,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,ComplexClass,37.82,38.88,38.96,40.70,45.55,44.61,44.38,46.14,45.40,45.22
